In [ ]:
from agents import Agent, trace, Runner,WebSearchTool,function_tool
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field
from agents import ModelSettings
from IPython.display import Markdown, display
from agents import Agent, WebSearchTool, trace, Runner, function_tool
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from typing import Dict

In [30]:
load_dotenv()

True

In [31]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### email agent

In [55]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("miiliab123@gmail.com") 
    to_email = To("iliasbessiness123456@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [56]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)


### prompt agent

In [39]:
prompt_INSTRUCTIONS=""" 
you are a prompt engener for product search agent. You will be given a text description of a product or product category,
or a specific product name,or eny field related to product.
 and you have to decide is the given text or prompt is goood or not  . 
 if not you have to  rewrite and enhance the prompt to be more specific and clear to get better results.
 your goal is to help user to get best prompt to give it to web search agent to get results from web search  tool.
"""

prompt_engennering_agent=Agent(name="web search agent"
      ,model="gpt-4o-mini",
      instructions=prompt_INSTRUCTIONS,
      model_settings=ModelSettings(tool_choice="auto")
)

### web search agent

In [43]:
WEB_INSTRUCTIONS=""" 
you are a product search agent. You will be given a product name
 and you have to find the best matching product from the web. 
 Use the web search tool to find the product and return the most relevant result
and find most rend products you can search chinees stores to find shepest ones that can be sell in usa.
required:
      1) if the user prompt is not related to product search you have to respond 
      with "i can only help you with product search"
      2)respond only with the prompt you will give to web search tool i want direct prompt
      3) if the user prompt is very general you have to enhance it to be more
"""

web_search_agent=Agent(name="web search agent"
      ,model="gpt-4o-mini",
      instructions=WEB_INSTRUCTIONS,
      tools=[WebSearchTool(search_context_size="low")],
      model_settings=ModelSettings(tool_choice="required")
)

### report agent

In [60]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [45]:
prompt="give 2 trend product in animals higeen  related to dogs cleaning"
with trace("retrive product info"):
    result=await Runner.run(prompt_engennering_agent,prompt)
display(Markdown(result.final_output))

The original prompt is unclear and has some grammatical issues. Here’s a revised version for better clarity and specificity: 

"Please provide two current trending products related to dog cleaning and hygiene."

In [58]:
prompt="give 2 trend product in animals higeen  related to dogs cleaning"
with trace("search for product"):
    results = await asyncio.gather(
        Runner.run(prompt_engennering_agent, prompt),
        Runner.run(web_search_agent, prompt),
        Runner.run(email_agent, prompt)
    )
for res in results:
    display(Markdown(res.final_output))

The original prompt is somewhat vague and lacks specificity, which may lead to unclear search results. Here’s a rewritten version:

---

**Revised Prompt:**

"Please provide two trending dog cleaning products that focus on animal hygiene, including details about their features, effectiveness, and customer reviews."

--- 

This version clarifies the request for trending products and specifies the need for features and customer feedback, which will yield more focused results.

Here are two trending products in the pet hygiene category related to dog cleaning:



**iRobot Roomba j7+ Pet Vacuum**  
This self-emptying robot vacuum is designed to detect and avoid pet messes, including accidents, while effectively cleaning up pet hair. It's ideal for maintaining a clean home in households with pets. ([petstalk.blog](https://petstalk.blog/best-pet-products-2025/?utm_source=openai))




**Shark StainForce Spot Cleaner**  
A compact, cordless spot cleaner that utilizes powerful suction and a dual-formula cleaning spray to eliminate both fresh and set-in stains on carpets, upholstery, and mattresses. It's particularly useful for homes with pets or children. ([tomsguide.com](https://www.tomsguide.com/home/home-appliances/i-just-tried-the-shark-stainforce-and-this-could-be-my-favorite-home-gadget-of-the-year?utm_source=openai))


These products are available through various retailers, including Amazon and Chewy. 

Here are two trending products related to dog hygiene and cleaning:

1. **Dog Bathing Systems**:
   - **Description**: These systems can include portable dog showers or specialized bathing tubs that make it easier to wash dogs at home. Many models have built-in water heaters and adjustable water pressure settings, ensuring a comfortable experience for pets.
   - **Trend**: Increasing popularity due to the convenience they offer for pet owners, especially those with larger or multiple dogs.

2. **Pet Wipes**:
   - **Description**: Antibacterial and hypoallergenic wipes designed specifically for pets. They are used for cleaning paws, fur, and sensitive areas without the need for a full bath. Many wipes are infused with natural ingredients that help combat odors and provide a quick clean.
   - **Trend**: Rising demand as pet owners look for quick and effective ways to maintain cleanliness between baths.

These products address common hygiene concerns and enhance the overall care of dogs.

In [ ]:
query="give 2 trend product in animals higeen  related to dogs cleaning"
if query:
    async def main():
            with trace("Research trace"):
                result = await Runner.run(prompt_engennering_agent, f"Query: {query}")
                input = f"Search term: {result.final_output}"
                result = await Runner.run(web_search_agent, input)
                input = f"Original query: {query}\nSummarized search results: {result.final_output}"
                result = await Runner.run(writer_agent, input)
                #st.spinner("Writing email...")
                result = await Runner.run(email_agent, result.markdown_report)
                print("Hooray!")
    asyncio.run(main())

Starting research...
Planning searches...
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


In [ ]:
result.follow_up_questions

['What technological advancements are being integrated into pet grooming tools?',
 'How do consumer purchasing patterns in pet accessories change with the introduction of multi-functional products?',
 'What other emerging trends can we anticipate in the pet hygiene and grooming industry?',
 'How do user reviews compare between traditional grooming tools and new integrated systems?',
 'What role does marketing play in the popularity of modern dog grooming products?']